# Constructing a Dobble card set

Every card of a Dobble game contains a fixed number $n$ of symbols. The set of cards is defined by the rules

1. any two cards have exactly one symbol in common
2. any two symbols appear together on exactly one card

It turns out that these requirements have an exact geometrical analogue. In two-dimensional affine geometry, the following two rules hold:

1. any two non-parallel lines meet in exactly one point
2. any two points lie on exactly one line

Since these characteristics depend only on the underlying vector-space, they particularly hold for vector-spaces over *finite fields* $\mathbb{F}_q$ with $q$ elements. 

So the strategy to construct a Dobble card game is as follows. Associate "card" with "line" and "symbol" with "point" and consider the affine plane over $\mathbb{F}_q$. 

Next, construct every line in the finite affine plane $A(\mathbb{F}_q)$. Consider therefore that every line is defined by an equation
$$
    \mathbf{w}\cdot\mathbf{x} + b = 0
$$
This means that every line is defined by a tuple $(\mathbf{w}, b)$ where $\mathbf{w}$ ranges over all different "directions" of the vector space. These are given by the set of the representatives 
$$
    (0,1)^T, (1,0)^T, (1,2)^T, \ldots, (1,n-1)^T,
$$
since obviously, these vectors are linearly independent, and for any other vector $(x,y) \in \mathbb{F}_q$ we have
$$
    (x,y)^T = \left\{ 
        \begin{array}{ll}
            x \cdot (1, y \cdot x^{-1})^T & \text{ für } x\not=0\\
            y \cdot (0,1)^T & \text{ für } x=0
        \end{array}
    \right.
    ,
$$
i.e. the vector $(x,y)^T$ is a multiple of one of the given vectors $w_i$. 

In [4]:
import random
import galois

class Line:
    def __init__(self, w, b, dim):
        self.w = w
        self.b = b
        self.dim = dim
        # The points on the line. 
        # All parallel lines (lines with equal w) have a "point at infinity" in common which, 
        # by convention, we denote with (-w_0, -w_1)
        self.points = [(-w[0],-w[1])]
        self.GF = galois.GF(dim)

    def contains(self, x):
        return (self.GF(self.w[0])*self.GF(x[0]) + self.GF(self.w[1])*self.GF(x[1]) + self.GF(self.b)) == 0      

    def print(self):
        print(f"w={self.w}x+{self.b}: Points: {self.points}")

class Dobble:

    def __init__(self, n):
        self.n = n
        # In a finite affine vector space over a field of order n there are:
        #   -- n+1 different directions: w=(0,1),(1,0),...(1,n-1)
        #   -- n different offsets: b=0,...,n-1
        # This makes n^2+n different lines defined by wx+b=0
        self.lines = [Line((0,1),b, n) for b in range(n)] + [Line((1,m),b,n) for m in range(n) for b in range(n)]
        self.points = [(x,y) for x in range(n) for y in range(n)]

        # Assign the points to the line
        for line in self.lines:
            for x in self.points:
                if line.contains(x):
                    line.points.append(x)
        
        # Add the "line at infinity" containing all "points at infinity"
        line_at_infinity = Line((0,0),0, n)
        line_at_infinity.points = [(0,-1)] + [(-1,-m) for m in range(n)]
        self.lines.append(line_at_infinity)

    def play(self):
        i,j = random.sample(range(len(self.lines)), 2)
        common_points = []
        for x in self.lines[i].points:
            for y in self.lines[j].points:
                if x==y:
                    common_points.append(x)
        
        print("New Game")
        self.lines[i].print()
        self.lines[j].print()
        print(F"Common point {common_points}")
        print("")

        
dobble = Dobble(7)

for i in range(10):
    dobble.play()


New Game
w=(1, 0)x+6: Points: [(-1, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6)]
w=(0, 0)x+0: Points: [(0, -1), (-1, 0), (-1, -1), (-1, -2), (-1, -3), (-1, -4), (-1, -5), (-1, -6)]
Common point [(-1, 0)]

New Game
w=(1, 2)x+1: Points: [(-1, -2), (0, 3), (1, 6), (2, 2), (3, 5), (4, 1), (5, 4), (6, 0)]
w=(1, 2)x+0: Points: [(-1, -2), (0, 0), (1, 3), (2, 6), (3, 2), (4, 5), (5, 1), (6, 4)]
Common point [(-1, -2)]

New Game
w=(1, 0)x+2: Points: [(-1, 0), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6)]
w=(1, 6)x+2: Points: [(-1, -6), (0, 2), (1, 3), (2, 4), (3, 5), (4, 6), (5, 0), (6, 1)]
Common point [(5, 0)]

New Game
w=(1, 6)x+3: Points: [(-1, -6), (0, 3), (1, 4), (2, 5), (3, 6), (4, 0), (5, 1), (6, 2)]
w=(1, 1)x+5: Points: [(-1, -1), (0, 2), (1, 1), (2, 0), (3, 6), (4, 5), (5, 4), (6, 3)]
Common point [(3, 6)]

New Game
w=(1, 0)x+0: Points: [(-1, 0), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6)]
w=(1, 0)x+4: Points: [(-1, 0), (3, 0), (3, 1), (3, 2), (3, 

In [27]:
a,b = random.sample(range(21), 2)

print(a)
print(b)

7
9
